In [2]:
# Import necessary libraries
import os
import torch
import numpy as np
import pandas as pd
import time
import json
from datetime import datetime
from PIL import Image
from torchvision import transforms
import timm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, make_scorer
)
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint, uniform
import xgboost as xgb
import joblib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.colors import LinearSegmentedColormap
from sklearn.metrics import RocCurveDisplay
import torch.nn as nn

# Set font properties for academic-style plots
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['figure.titlesize'] = 16

# Record script start time
start_time_script = time.time()

# Global Configuration
ROOT_DIR = "D:\\iate_project\\data\\"
DL_MODEL_DIR = os.path.join(ROOT_DIR, "results", "dl_output", "dl_training", "models")
HYBRID_OUTPUT_DIR = os.path.join(ROOT_DIR, "results", "hybrid_output")
HYBRID_FEATURE_DIR = os.path.join(HYBRID_OUTPUT_DIR, "features")
HYBRID_RESULTS_DIR = os.path.join(HYBRID_OUTPUT_DIR, "results")
HYBRID_MODELS_DIR = os.path.join(HYBRID_RESULTS_DIR, "models")
HYBRID_PLOTS_DIR = os.path.join(HYBRID_RESULTS_DIR, "plots")

# Path to original dataset
DATASET_PATH = os.path.join(ROOT_DIR, "raw")

# Create necessary directories
for d in [HYBRID_OUTPUT_DIR, HYBRID_FEATURE_DIR, HYBRID_RESULTS_DIR, HYBRID_MODELS_DIR, HYBRID_PLOTS_DIR]:
    os.makedirs(d, exist_ok=True)

# Hyperparameter tuning settings
N_ITER_SEARCH = 15  # Number of parameter settings sampled in RandomizedSearchCV
CV_FOLDS = 5        # Number of cross-validation folds for hyperparameter tuning

# Define base classifiers for hyperparameter optimization
BASE_CLASSIFIERS = {
    'SVM': {
        'model': SVC(probability=True, random_state=42, cache_size=1000),
        'param_dist': {
            'C': uniform(50, 200),
            'gamma': ['scale', 'auto', 0.001, 0.01, 0.1],
            'kernel': ['rbf', 'poly']
        },
        'expects_proba': True,
    },
    'RandomForest': {
        'model': RandomForestClassifier(n_jobs=-1, random_state=42),
        'param_dist': {
            'n_estimators': randint(100, 500),
            'max_depth': [None, 10, 20, 30, 40, 50],
            'min_samples_split': randint(2, 20),
            'min_samples_leaf': randint(1, 10),
            'max_features': ['sqrt', 'log2', None]
        },
        'expects_proba': True,
    },
    'XGBoost': {
        'model': xgb.XGBClassifier(objective='binary:logistic', random_state=42, n_jobs=-1),
        'param_dist': {
            'n_estimators': randint(100, 500),
            'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
            'max_depth': randint(3, 10),
            'subsample': uniform(0.6, 0.4),
            'colsample_bytree': uniform(0.6, 0.4)
        },
        'expects_proba': True,
    },
    'ExtraTrees': {
        'model': ExtraTreesClassifier(n_jobs=-1, random_state=42),
        'param_dist': {
            'n_estimators': randint(100, 600),
            'max_depth': [None, 10, 20, 30, 40, 50],
            'min_samples_split': randint(2, 20),
            'min_samples_leaf': randint(1, 10),
            'max_features': ['sqrt', 'log2', None]
        },
        'expects_proba': True
    },
    'KNN': {
        'model': KNeighborsClassifier(n_jobs=-1),
        'param_dist': {
            'n_neighbors': randint(3, 15),
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan', 'minkowski']
        },
        'expects_proba': True,
    },
    'DecisionTree': {
        'model': DecisionTreeClassifier(random_state=42),
        'param_dist': {
            'max_depth': [None, 10, 20, 30, 40, 50],
            'min_samples_split': randint(2, 20),
            'min_samples_leaf': randint(1, 10),
            'criterion': ['gini', 'entropy']
        },
        'expects_proba': True
    }
}

print("HYBRID APPROACH: DEEP FEATURES + ML CLASSIFIERS")

#-------------------------------------------------------------
# STAGE 1: DEEP FEATURE EXTRACTION
#-------------------------------------------------------------
print("STAGE 1: DEEP FEATURE EXTRACTION")

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the mobilevit_xs model (best model from DL experiment)
model_path = os.path.join(DL_MODEL_DIR, "mobilevit_xs_final_model.pth")
model = timm.create_model('mobilevit_xs', pretrained=False, num_classes=2)
model.load_state_dict(torch.load(model_path, map_location=device))

# Create a feature extractor by removing the classification head
class FeatureExtractor(nn.Module):
    def __init__(self, model):
        super(FeatureExtractor, self).__init__()
        # Get all layers except the final classifier
        self.features = nn.Sequential(*list(model.children())[:-1])

    def forward(self, x):
        # Get features
        x = self.features(x)
        # Global average pooling
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1))
        # Flatten
        x = torch.flatten(x, 1)
        return x

# Create feature extractor and move to device
feature_extractor = FeatureExtractor(model)
feature_extractor.to(device)
feature_extractor.eval()  # Set to evaluation mode

# Define image transformation (same as used in DL training)
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Lists to hold extracted data
features_list = []
labels = []
paths = []
splits = []

# Define train/test split ratio (70% train, 30% test)
train_ratio = 0.7

extraction_start_time = time.time()

# Create storage for class distribution info
class_counts = {'train': {'normal': 0, 'defect': 0},
               'test': {'normal': 0, 'defect': 0}}

# Process image directories
classes = ['normal', 'defect']
processing_types = ['dry', 'honey', 'wet']
roast_levels = ['dark', 'light', 'medium']

# Count total files for progress tracking
total_files_count = 0
for class_name in classes:
    for proc_type in processing_types:
        for roast in roast_levels:
            subfolder_path = os.path.join(DATASET_PATH, class_name, proc_type, roast)
            if os.path.exists(subfolder_path):
                image_files = [f for f in os.listdir(subfolder_path)
                             if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                total_files_count += len(image_files)

print(f"Total images to process: {total_files_count}")

# Process images
with torch.no_grad():  # No gradient computation needed
    for class_name in classes:
        for proc_type in processing_types:
            for roast in roast_levels:
                # Construct the path to the current subfolder
                subfolder_path = os.path.join(DATASET_PATH, class_name, proc_type, roast)

                if not os.path.exists(subfolder_path):
                    continue

                # Get list of image files in the current subfolder
                image_files = [f for f in os.listdir(subfolder_path)
                              if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

                if not image_files:
                    continue

                # Shuffle files for random split
                np.random.seed(42)  # For reproducibility
                np.random.shuffle(image_files)

                # Split into train and test
                split_idx = int(len(image_files) * train_ratio)
                train_files = image_files[:split_idx]
                test_files = image_files[split_idx:]

                # Update class distribution counts
                class_counts['train'][class_name] += len(train_files)
                class_counts['test'][class_name] += len(test_files)

                # Process each split
                for split_name, files in [('train', train_files), ('test', test_files)]:
                    batch_size = len(files)

                    # Process each image
                    for file_ in files:
                        # Construct full path to the image
                        img_path = os.path.join(subfolder_path, file_)

                        try:
                            # Open and preprocess image
                            img = Image.open(img_path).convert('RGB')
                            img_tensor = transform(img).unsqueeze(0).to(device)

                            # Extract features
                            feat = feature_extractor(img_tensor)

                            # Add to lists
                            features_list.append(feat.cpu().numpy().flatten())
                            labels.append(class_name)
                            paths.append(os.path.join(class_name, proc_type, roast, file_))
                            splits.append(split_name)

                        except Exception as e:
                            pass

# Create DataFrame
feature_dim = features_list[0].shape[0]
feat_names = [f'deep_feature_{i}' for i in range(feature_dim)]

df = pd.DataFrame(features_list, columns=feat_names)
df['label'] = labels
df['image_path'] = paths
df['split'] = splits

# Split and scale
train_df = df[df['split'] == 'train']
test_df = df[df['split'] == 'test']
scaler = StandardScaler()
train_mat = scaler.fit_transform(train_df[feat_names])
test_mat = scaler.transform(test_df[feat_names])
df.loc[df['split'] == 'train', feat_names] = train_mat
df.loc[df['split'] == 'test', feat_names] = test_mat

# Save scaler info
sc_info = {
    'mean': scaler.mean_.tolist(),
    'scale': scaler.scale_.tolist(),
    'feature_names': feat_names
}
with open(os.path.join(HYBRID_FEATURE_DIR, 'scaler_params.json'), 'w') as f:
    json.dump(sc_info, f, indent=4)

# Save CSV of extracted features
output_csv_path = os.path.join(HYBRID_FEATURE_DIR, 'deep_features.csv')
df.to_csv(output_csv_path, index=False)

extraction_time = time.time() - extraction_start_time

# Build metadata
class_dist = df.groupby(['split', 'label']).size()
class_dist_dict = {
    f"{sp}_{lb}": int(cnt)
    for (sp, lb), cnt in class_dist.items()
}

meta = {
    'extraction_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'processing_time': {
        'total_time': f"{extraction_time:.2f} seconds",
        'per_image': f"{extraction_time/len(df):.4f} seconds/image"
    },
    'dataset_statistics': {
        'total_samples': len(df),
        'class_distribution': class_dist_dict,
        'feature_dimensions': feature_dim
    },
    'model_info': {
        'architecture': 'mobilevit_xs',
        'feature_layer': 'before_classifier'
    }
}

meta_path = os.path.join(HYBRID_FEATURE_DIR, 'extraction_metadata.json')
with open(meta_path, 'w') as f:
    json.dump(meta, f, indent=4)

# Print extraction summary
print("DEEP FEATURE EXTRACTION COMPLETE")
print(f"Total samples: {len(df)} images")
print(f"Feature dimensionality: {feature_dim} features")
print(f"Processing time: {extraction_time:.1f} seconds ({len(df)/extraction_time:.1f} images/sec)")

#-------------------------------------------------------------
# STAGE 2: TRAIN ML CLASSIFIERS ON DEEP FEATURES
#-------------------------------------------------------------
print("STAGE 2: TRAIN ML CLASSIFIERS ON DEEP FEATURES")

# Prepare data for classification
feat_cols = [c for c in df.columns if c.startswith('deep_feature_')]
train_df = df[df['split'] == 'train']
test_df = df[df['split'] == 'test']

X_train = train_df[feat_cols].values
X_test = test_df[feat_cols].values
label_map = {'normal': 0, 'defect': 1}
y_train = np.array([label_map[l] for l in train_df['label']])
y_test = np.array([label_map[l] for l in test_df['label']])

# Setup for results collection
all_results = {}
best_model_info = {
    'score': -1,
    'clf_name': None,
    'model': None,
    'results': None,
    'best_params': None
}

# Evaluate each classifier
for clf_name, clf_info in BASE_CLASSIFIERS.items():
    eval_start = time.time()

    # Create a pipeline with the classifier
    pipeline = clf_info['model']

    # Setup parameter distributions
    param_dist = clf_info['param_dist']

    # Create RandomizedSearchCV for hyperparameter tuning
    random_search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_dist,
        n_iter=N_ITER_SEARCH,
        cv=CV_FOLDS,
        scoring='f1',
        n_jobs=-1,
        random_state=42,
        verbose=0
    )

    # Fit RandomizedSearchCV to find best parameters
    random_search.fit(X_train, y_train)

    # Get best model and parameters
    best_model = random_search.best_estimator_
    best_params = random_search.best_params_

    # Cross-validation with best model
    cv_metrics = {}
    splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    scoring = {
        'accuracy': 'accuracy',
        'precision': make_scorer(precision_score, zero_division=0),
        'recall': 'recall',
        'f1': 'f1',
        'roc_auc': 'roc_auc'
    }
    cv_results = cross_validate(
        best_model, X_train, y_train,
        scoring=scoring, cv=splitter, n_jobs=-1, return_train_score=True
    )

    # Process CV results
    for metric, scores in [(k.replace('test_', ''), cv_results[k])
                        for k in cv_results if k.startswith('test_')]:
        cv_metrics[metric] = {
            'mean': float(scores.mean()),
            'std': float(scores.std()),
            'values': scores.tolist()
        }

    # Predict test set
    y_pred = best_model.predict(X_test)
    y_proba = None
    if clf_info['expects_proba']:
        y_proba = best_model.predict_proba(X_test)[:, 1]

    # Compute test metrics
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'roc_auc': roc_auc_score(y_test, y_proba) if y_proba is not None else None
    }

    # Get feature importances if available
    feat_imp = None
    if hasattr(best_model, 'feature_importances_'):
        feat_imp = best_model.feature_importances_
    elif hasattr(best_model, 'coef_'):
        feat_imp = np.abs(best_model.coef_[0])

    # Record total time
    total_t = time.time() - eval_start

    # Store results
    result_dict = {
        'model_name': clf_name,
        'cross_validation': cv_metrics,
        'test_performance': metrics,
        'timing': {
            'total_time': total_t
        },
        'feature_importance': feat_imp.tolist() if feat_imp is not None else None,
        'best_params': best_params
    }
    all_results[clf_name] = result_dict

    # Track the best model by F1 score
    cur_f1 = metrics['f1']
    if cur_f1 > best_model_info['score']:
        best_model_info['score'] = cur_f1
        best_model_info['clf_name'] = clf_name
        best_model_info['model'] = best_model
        best_model_info['results'] = result_dict
        best_model_info['best_params'] = best_params

    # Print result with clear formatting
    is_best = (best_model_info['clf_name'] == clf_name)
    best_marker = "★ " if is_best else "  "

    print(f"{best_marker}{clf_name:15} | "
          f"F1: {metrics['f1']:.4f} | "
          f"Acc: {metrics['accuracy']:.4f} | "
          f"Prec: {metrics['precision']:.4f} | "
          f"Rec: {metrics['recall']:.4f}")

# Save results as JSON
out_json = os.path.join(HYBRID_RESULTS_DIR, 'classification_results.json')
with open(out_json, 'w') as f:
    json.dump(all_results, f, indent=4)

# Save best model
model_path = os.path.join(HYBRID_MODELS_DIR, 'best_model.joblib')
joblib.dump(best_model_info['model'], model_path)

# Save a detailed report
best_report = {
    'model_summary': {
        'name': best_model_info['clf_name'],
        'feature_type': 'deep_features',
        'total_features': len(feat_cols),
        'best_hyperparameters': best_model_info['best_params']
    },
    'performance_metrics': best_model_info['results']['test_performance'],
    'timing_information': best_model_info['results']['timing']
}
report_path = os.path.join(HYBRID_RESULTS_DIR, 'detailed_report.json')
with open(report_path, 'w') as f:
    json.dump(best_report, f, indent=4)

#-------------------------------------------------------------
# ENHANCED VISUALIZATIONS FOR ACADEMIC PUBLICATION
#-------------------------------------------------------------

# 1. Enhanced Confusion Matrix
cm_path = os.path.join(HYBRID_PLOTS_DIR, 'best_model_confusion_matrix.png')
cm = confusion_matrix(y_test, best_model_info['model'].predict(X_test))
classes = ['Normal', 'Defect']  # Capitalize class names for publication
total_samples = np.sum(cm)

# Calculate percentages for annotations
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
tn, fp, fn, tp = cm.ravel()

# Create custom colormap for better visualization
colors = ["#f7fbff", "#08306b"]  # From light blue to dark blue
cmap = LinearSegmentedColormap.from_list("custom_blues", colors, N=100)

fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

# Add title and axis labels
ax.set_title(f'Confusion Matrix - {best_model_info["clf_name"]} with Deep Features',
             fontweight='bold', pad=20)
ax.set_xlabel('Predicted Label', fontweight='bold', labelpad=15)
ax.set_ylabel('True Label', fontweight='bold', labelpad=15)

# Add tick marks and labels
ax.set_xticks(np.arange(len(classes)))
ax.set_yticks(np.arange(len(classes)))
ax.set_xticklabels(classes)
ax.set_yticklabels(classes)

# Rotate the tick labels and set alignment
plt.setp(ax.get_xticklabels(), rotation=0, ha="center", rotation_mode="anchor")

# Loop over data dimensions and create text annotations with counts and percentages
thresh = cm.max() / 2.0
for i in range(len(classes)):
    for j in range(len(classes)):
        if i == j:
            color = "white"  # For diagonal elements (correctly classified)
        else:
            color = "black"

        # Format as count (percentage)
        percentage = cm_norm[i, j] * 100
        text = f"{cm[i, j]}\n({percentage:.1f}%)"

        ax.text(j, i, text, ha="center", va="center",
                color=color if cm[i, j] > thresh else "black",
                fontsize=14, fontweight='bold')

# Add overall accuracy and F1 score in the figure
accuracy = (tn + tp) / total_samples
f1 = best_model_info['results']['test_performance']['f1']
precision = best_model_info['results']['test_performance']['precision']
recall = best_model_info['results']['test_performance']['recall']

# Add performance metrics as text box
textstr = '\n'.join((
    f'Accuracy: {accuracy:.4f}',
    f'Precision: {precision:.4f}',
    f'Recall: {recall:.4f}',
    f'F1 Score: {f1:.4f}'))

props = dict(boxstyle='round', facecolor='white', alpha=0.8)
ax.text(1.05, 0.5, textstr, transform=ax.transAxes, fontsize=12,
        verticalalignment='center', bbox=props)

# Final formatting
fig.tight_layout()
plt.grid(False)
plt.savefig(cm_path, dpi=300, bbox_inches='tight')
plt.close()

# 2. Enhanced ROC Curve
if best_model_info['results']['test_performance']['roc_auc'] is not None:
    roc_path = os.path.join(HYBRID_PLOTS_DIR, 'best_model_roc_curve.png')
    y_proba = best_model_info['model'].predict_proba(X_test)[:, 1]

    fig, ax = plt.subplots(figsize=(10, 8))

    # Plot ROC curve
    viz = RocCurveDisplay.from_predictions(
        y_test, y_proba,
        name=f'{best_model_info["clf_name"]}',
        ax=ax,
        plot_chance_level=True  # Add diagonal reference line
    )

    # Get AUC value
    auc_value = best_model_info['results']['test_performance']['roc_auc']

    # Add title and improve labels
    ax.set_title(f'ROC Curve - {best_model_info["clf_name"]} with Deep Features',
                 fontweight='bold', pad=20)
    ax.set_xlabel('False Positive Rate', fontweight='bold', labelpad=15)
    ax.set_ylabel('True Positive Rate', fontweight='bold', labelpad=15)

    # Add AUC value to legend
    handles, labels = ax.get_legend_handles_labels()
    labels = [f'{best_model_info["clf_name"]} (AUC = {auc_value:.4f})', 'Chance level (AUC = 0.5)']
    ax.legend(handles, labels, loc='lower right')

    # Add grid for better readability
    ax.grid(True, linestyle='--', alpha=0.7)

    # Format axes with percentage labels
    ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

    # Add operating points
    thresholds = [0.3, 0.5, 0.7, 0.9]
    for threshold in thresholds:
        y_pred_thresh = (y_proba >= threshold).astype(int)
        tn_t, fp_t, fn_t, tp_t = confusion_matrix(y_test, y_pred_thresh).ravel()
        tpr = tp_t / (tp_t + fn_t)
        fpr = fp_t / (fp_t + tn_t)

        # Only plot if point is within axis limits
        if 0 <= fpr <= 1 and 0 <= tpr <= 1:
            ax.plot(fpr, tpr, 'ro', markersize=8)
            ax.annotate(f'T={threshold:.1f}',
                        xy=(fpr, tpr),
                        xytext=(fpr+0.05, tpr-0.05),
                        fontsize=10,
                        arrowprops=dict(arrowstyle='->', color='red'))

    fig.tight_layout()
    plt.savefig(roc_path, dpi=300, bbox_inches='tight')
    plt.close()

# 3. Enhanced Performance Comparison Bar Chart
try:
    # Load previous approach results
    ml_results_path = os.path.join(ROOT_DIR, "results", "ml_output", "ml_classification_results", "detailed_report.json")
    with open(ml_results_path, 'r') as f:
        ml_results = json.load(f)

    dl_results_path = os.path.join(ROOT_DIR, "results", "dl_output", "dl_training", "results", "final_model_comparison.json")
    with open(dl_results_path, 'r') as f:
        dl_results = json.load(f)

    # Get best DL model metrics
    best_dl_model = dl_results['best_model']
    best_dl_metrics = dl_results['models'][best_dl_model]

    # Create comparison data
    comparison = {
        'Pure ML\n(Handcrafted Features + SVM)': {
            'accuracy': float(ml_results['performance_metrics']['accuracy']),
            'precision': float(ml_results['performance_metrics']['precision']),
            'recall': float(ml_results['performance_metrics']['recall']),
            'f1_score': float(ml_results['performance_metrics']['f1']),
        },
        'Hybrid\n(Deep Features + ML)': {
            'accuracy': float(best_model_info['results']['test_performance']['accuracy']),
            'precision': float(best_model_info['results']['test_performance']['precision']),
            'recall': float(best_model_info['results']['test_performance']['recall']),
            'f1_score': float(best_model_info['results']['test_performance']['f1']),
        },
        'Pure DL\n(End-to-End Deep Learning)': {
            'accuracy': float(best_dl_metrics['accuracy']),
            'precision': float(best_dl_metrics['precision']),
            'recall': float(best_dl_metrics['recall']),
            'f1_score': float(best_dl_metrics['f1_score']),
        }
    }

    # Save comparison as JSON
    comparison_path = os.path.join(HYBRID_RESULTS_DIR, 'approach_comparison.json')
    with open(comparison_path, 'w') as f:
        json.dump(comparison, f, indent=4)

    # Enhanced comparison plot
    plot_metrics = ['accuracy', 'precision', 'recall', 'f1_score']
    metric_labels = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
    approaches = list(comparison.keys())

    # High-quality colors for approaches (colorblind-friendly)
    colors = ['#377eb8', '#ff7f00', '#4daf4a']

    fig, ax = plt.subplots(figsize=(12, 8))
    x = np.arange(len(plot_metrics))
    width = 0.25

    # Plot bars for each approach with enhanced colors and edges
    for i, (approach, color) in enumerate(zip(approaches, colors)):
        values = [comparison[approach][m] for m in plot_metrics]
        bars = ax.bar(x + i*width, values, width, label=approach,
                     color=color, edgecolor='black', linewidth=1.5, alpha=0.85)

        # Add value labels on top of bars
        for bar, value in zip(bars, values):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{value:.3f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

    # Add horizontal grid lines for better readability
    ax.yaxis.grid(True, linestyle='--', alpha=0.7)

    # Customize axes
    ax.set_ylabel('Score', fontsize=16, fontweight='bold')
    ax.set_title('Performance Comparison of Different Classification Approaches',
                fontsize=18, fontweight='bold', pad=20)
    ax.set_xticks(x + width)
    ax.set_xticklabels(metric_labels, fontweight='bold')

    # Format y-axis as percentage
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

    # Set y-axis limits for better visualization
    ax.set_ylim(0.7, 1.01)  # Start from 0.7 to better show differences

    # Enhance legend
    ax.legend(title='Approach', title_fontsize=14,
              loc='lower left', fontsize=12, framealpha=0.9, edgecolor='black')

    # Add annotations highlighting the best approach
    best_approach = max(approaches, key=lambda x: comparison[x]['f1_score'])
    best_f1 = comparison[best_approach]['f1_score']
    ax.annotate(f'Best F1: {best_f1:.4f}',
                xy=(3 + width, best_f1),  # Position at the F1 score bar
                xytext=(3 + width, best_f1 + 0.05),  # Text position above bar
                arrowprops=dict(arrowstyle='->', color='red'),
                fontsize=12, fontweight='bold', color='red',
                ha='center')

    # Improve overall appearance
    fig.tight_layout()
    plt.savefig(os.path.join(HYBRID_PLOTS_DIR, 'approach_comparison.png'), dpi=300, bbox_inches='tight')
    plt.close()

    # Print comparison table
    print("\nPerformance Comparison Table:")
    print(f"{'Approach':40} | {'Accuracy':10} | {'Precision':10} | {'Recall':10} | {'F1 Score':10}")

    for approach in approaches:
        print(f"{approach:40} | "
              f"{comparison[approach]['accuracy']:.4f}     | "
              f"{comparison[approach]['precision']:.4f}     | "
              f"{comparison[approach]['recall']:.4f}     | "
              f"{comparison[approach]['f1_score']:.4f}")

    # Determine best approach
    best_approach = max(approaches, key=lambda x: comparison[x]['f1_score'])
    print(f"\nBest Approach Based on F1 Score: {best_approach}")
    print(f"F1 Score: {comparison[best_approach]['f1_score']:.4f}")

except Exception as e:
    print("Comparison with previous approaches not available.")

# Calculate total script execution time
total_time_script = time.time() - start_time_script

# Print final summary
print("\nHYBRID APPROACH COMPLETED")
print(f"Best Model: {best_model_info['clf_name']}")
print(f"F1 Score: {best_model_info['score']:.4f}")
print(f"Accuracy: {best_model_info['results']['test_performance']['accuracy']:.4f}")
print(f"Total execution time: {total_time_script/60:.1f} minutes")

HYBRID APPROACH: DEEP FEATURES + ML CLASSIFIERS
STAGE 1: DEEP FEATURE EXTRACTION
Total images to process: 5400
DEEP FEATURE EXTRACTION COMPLETE
Total samples: 5400 images
Feature dimensionality: 384 features
Processing time: 61.6 seconds (87.6 images/sec)
STAGE 2: TRAIN ML CLASSIFIERS ON DEEP FEATURES
★ SVM             | F1: 0.9478 | Acc: 0.9660 | Prec: 0.9727 | Rec: 0.9241
★ RandomForest    | F1: 0.9532 | Acc: 0.9691 | Prec: 0.9640 | Rec: 0.9426
  XGBoost         | F1: 0.9531 | Acc: 0.9691 | Prec: 0.9658 | Rec: 0.9407
  ExtraTrees      | F1: 0.9522 | Acc: 0.9685 | Prec: 0.9639 | Rec: 0.9407
  KNN             | F1: 0.9491 | Acc: 0.9667 | Prec: 0.9673 | Rec: 0.9315
  DecisionTree    | F1: 0.9242 | Acc: 0.9488 | Prec: 0.9117 | Rec: 0.9370

Performance Comparison Table:
Approach                                 | Accuracy   | Precision  | Recall     | F1 Score  
Pure ML
(Handcrafted Features + SVM)     | 0.9117     | 0.9144     | 0.8111     | 0.8597
Hybrid
(Deep Features + ML)             